In [ ]:
% matplotlib inline

from __future__ import division

import glob
import os
import numpy as np
from collections import namedtuple
from fractions import Fraction

import colour
from colour.plotting import *

from colour_demosaicing import (
    demosaicing_CFA_Bayer_bilinear,
    demosaicing_CFA_Bayer_Malvar2004,
    demosaicing_CFA_Bayer_DDFAPD)
from colour_hdri.exif import get_exif_data, ExposureData
from colour_hdri.image import Image
from colour_hdri.merge import merge_to_hdri
from colour_hdri.weighting_functions import (
    hat_function,
    normal_distribution_function,
    weighting_function_Debevec1997)


DIRECTORY = '/Users/kelsolaar/Documents/Development/colour-science/colour-hdri/resources/NIKON D800/Shadows_001/'

OECF = colour.RGB_COLOURSPACES['sRGB'].transfer_function

def custom_weight_function(a):
    a = np.asarray(a)

    return np.where(a < 0.5,
                   hat_function(a),
                   1)


def process(dng_files, 
            output_directory,
            batch_size=9, 
            black_level=None, 
            white_level=None, 
            post_demosaicing=False,
            weight_function=custom_weight_function):

    for dng_files in colour.batch(dng_files, batch_size):
        images = []
        for dng_file in dng_files:
            print('Reading "{}"...'.format(dng_file))
            exif_data = get_exif_data(dng_file)
            aperture = float(exif_data['EXIF']['Aperture Value'][0])
            shutter_speed = float(Fraction(
                exif_data['EXIF']['Shutter Speed Value'][0]))
            iso = float(exif_data['EXIF']['ISO'][0])

            images.append(Image(dng_file,
                                colour.read_image(dng_file.replace('dng', 'tiff')),
                                ExposureData(aperture, shutter_speed, iso)))

        if post_demosaicing:
            if black_level is None:
                black_level = 0

            if white_level is None:
                white_level = 0
                for image in images:
                    white_level = max(white_level, np.max(image.data))

            for image in images:
                print('Scaling "{}"...'.format(image.path))
                image.pixel_data -= black_level
                image.pixel_data *= 1 / white_level

        path = os.path.join(
            output_directory,
            '{0}_{1}.{2}'.format(
                os.path.splitext(os.path.basename(images[0].path))[0],
                batch_size,
                'exr'))

        print('Merging "{0}"...'.format(path))
        image = merge_to_hdri(images, custom_weight_function)

        if post_demosaicing:
            print('Demosaicing "{0}"...'.format(path))
            image = demosaicing_CFA_Bayer_bilinear(image)
        
        print('Writing "{0}"...'.format(path))
        colour.write_image(image, path)
        


DNG_FILES = glob.glob('{0}/*.dng'.format(DIRECTORY))

process(DNG_FILES, DIRECTORY)

Reading "/Users/kelsolaar/Documents/Development/colour-science/colour-hdri/resources/NIKON D800/Shadows_001/DSC_2211.dng"...
Reading "/Users/kelsolaar/Documents/Development/colour-science/colour-hdri/resources/NIKON D800/Shadows_001/DSC_2212.dng"...
Reading "/Users/kelsolaar/Documents/Development/colour-science/colour-hdri/resources/NIKON D800/Shadows_001/DSC_2213.dng"...
Reading "/Users/kelsolaar/Documents/Development/colour-science/colour-hdri/resources/NIKON D800/Shadows_001/DSC_2214.dng"...
Reading "/Users/kelsolaar/Documents/Development/colour-science/colour-hdri/resources/NIKON D800/Shadows_001/DSC_2215.dng"...
Reading "/Users/kelsolaar/Documents/Development/colour-science/colour-hdri/resources/NIKON D800/Shadows_001/DSC_2216.dng"...
Reading "/Users/kelsolaar/Documents/Development/colour-science/colour-hdri/resources/NIKON D800/Shadows_001/DSC_2217.dng"...
Reading "/Users/kelsolaar/Documents/Development/colour-science/colour-hdri/resources/NIKON D800/Shadows_001/DSC_2218.dng"...


In [ ]:
import pylab

pylab.plot(np.linspace(0, 1, 100), hat_function(np.linspace(0, 1, 100)), label='Hat')
pylab.plot(np.linspace(0, 1, 100), normal_distribution_function(np.linspace(0, 1, 100)), label='Normal')
pylab.plot(np.linspace(0, 1, 100), weighting_function_Debevec1997(np.linspace(0, 1, 100)), label='Debevec (1997)')
pylab.plot(np.linspace(0, 1, 1000), custom_weight_function(np.linspace(0, 1, 1000)), label='Custom')

pylab.show()